In [2]:
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
import get_data
import plotly.express as px
import sqlalchemy
from sqlalchemy import create_engine, text
import api_info
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from sklearn.preprocessing import MinMaxScaler

c:\Users\jeurc\Anaconda3\envs\phylotorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set up database connection
username = api_info.USERNAME
password = api_info.PASSWORD
host = api_info.HOST
port = api_info.PORT
db_name = api_info.DB_NAME
endpoint = 'postgresql://' + username + ':' + password + '@' + host + ':' + port + '/' + db_name
engine = sqlalchemy.create_engine(endpoint)

try:
    with engine.connect() as connection:
        print('Database connection successful')
except Exception as e:
    print(e)

Database connection successful


In [4]:
# Load data from the database
df = pd.DataFrame(engine.connect().execute(text('SELECT datadate,gvkey,tic,cusip,dlttq,lseq,seqq,ltq,actq,invtq,lctq,oiadpq,saleq FROM quarterly')))
df = df.query('seqq != 0 & dlttq > 0 & actq > 0 & invtq > 0 & lctq > 0 & oiadpq > 0 & saleq > 0')

In [5]:
# Calculate signal columns
df['debt_to_equity'] = df['dlttq'] / df['seqq']
df['quick_ratio'] = (df['actq'] - df['invtq']) / df['lctq']
df['op_margin'] = df['oiadpq'] / df['saleq']

In [6]:
# Load returns data
returns_df = pd.DataFrame(engine.connect().execute(text('SELECT date,cusip,ticker,ret,prc,askhi,bidlo FROM returns')))
returns_df['ret'] = pd.to_numeric(returns_df['ret'], errors='coerce')
returns_df.dropna(subset=['ret'], inplace=True)

In [47]:
# Normalize and combine signals
scaler = MinMaxScaler()
df[['debt_to_equity_norm', 'quick_ratio_norm', 'op_margin_norm']] = scaler.fit_transform(df[['debt_to_equity', 'quick_ratio', 'op_margin']])

weights = {'debt_to_equity': 0.4, 'quick_ratio': 0.3, 'op_margin': 0.3}
df['weighted_sum'] = df['debt_to_equity_norm'] * weights['debt_to_equity'] + \
                     df['quick_ratio_norm'] * weights['quick_ratio'] + \
                     df['op_margin_norm'] * weights['op_margin']

In [23]:
# Calculate expanding quantiles outside the strategy and add them to the dataframe
quantile_80 = df['weighted_sum'].expanding().quantile(0.8)
quantile_20 = df['weighted_sum'].expanding().quantile(0.2)
df['quantile_80'] = quantile_80
df['quantile_20'] = quantile_20

# Merge the df and returns_df dataframes
merged_df = pd.merge(df, returns_df, left_on=['datadate', 'tic'], right_on=['date', 'ticker'])
merged_df = merged_df.sort_values(by='date')
merged_df = merged_df.set_index('date')

In [29]:
# Add an 'Open' column (assuming the open price is the same as the close price of the previous day)
merged_df['Open'] = merged_df['Close'].shift(1)

# Drop rows with missing data
merged_df.dropna(subset=['Open', 'High', 'Low', 'Close', 'quantile_80', 'quantile_20'], inplace=True)

In [49]:
class CombinedSignalStrategy(Strategy):
    def init(self):
        self.buy_signal = self.I(lambda: self.data.weighted_sum >= self.data.quantile_80)
        self.sell_signal = self.I(lambda: self.data.weighted_sum <= self.data.quantile_20)

    def next(self):
        if crossover(self.data.weighted_sum, self.data.quantile_80):
            self.buy()
        elif crossover(self.data.quantile_20, self.data.weighted_sum):
            self.sell()

In [71]:
from backtesting import Backtest, Strategy
from backtesting.test import GOOG

class SimplifiedStrategy(Strategy):
    def init(self):
        self.threshold = 700

    def next(self):
        if self.data.Close[-1] < self.threshold:
            print(f"Buy signal generated at {self.data.index[-1]}")
            self.buy()  # Adjust size and stop_loss as needed
        elif self.data.Close[-1] > self.threshold:
            print(f"Sell signal generated at {self.data.index[-1]}")
            self.sell()  # Adjust size and stop_loss as needed

# Run the backtest with the simplified strategy and GOOG data
bt = Backtest(GOOG, SimplifiedStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)

# Generate Sharpe ratio and PnL graph
print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
bt.plot()

Buy signal generated at 2004-08-20 00:00:00
Buy signal generated at 2004-08-23 00:00:00
Buy signal generated at 2004-08-24 00:00:00
Buy signal generated at 2004-08-25 00:00:00
Buy signal generated at 2004-08-26 00:00:00
Buy signal generated at 2004-08-27 00:00:00
Buy signal generated at 2004-08-30 00:00:00
Buy signal generated at 2004-08-31 00:00:00
Buy signal generated at 2004-09-01 00:00:00
Buy signal generated at 2004-09-02 00:00:00
Buy signal generated at 2004-09-03 00:00:00
Buy signal generated at 2004-09-07 00:00:00
Buy signal generated at 2004-09-08 00:00:00
Buy signal generated at 2004-09-09 00:00:00
Buy signal generated at 2004-09-10 00:00:00
Buy signal generated at 2004-09-13 00:00:00
Buy signal generated at 2004-09-14 00:00:00
Buy signal generated at 2004-09-15 00:00:00
Buy signal generated at 2004-09-16 00:00:00
Buy signal generated at 2004-09-17 00:00:00
Buy signal generated at 2004-09-20 00:00:00
Buy signal generated at 2004-09-21 00:00:00
Buy signal generated at 2004-09-

c:\Users\jeurc\Anaconda3\envs\phylotorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\jeurc\Anaconda3\envs\phylotorch\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\jeurc\Anaconda3\envs\phylotorch\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\jeurc\Anaconda3\envs\phylotorch\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


In [74]:
# Load data from the database
df = pd.DataFrame(engine.connect().execute(text('SELECT datadate,gvkey,tic,cusip,dlttq,lseq,seqq,ltq,actq,invtq,lctq,oiadpq,saleq FROM quarterly')))
df = df.query('seqq != 0 & dlttq > 0 & actq > 0 & invtq > 0 & lctq > 0 & oiadpq > 0 & saleq > 0')

# Calculate signal columns
df['debt_to_equity'] = df['dlttq'] / df['seqq']
df['quick_ratio'] = (df['actq'] - df['invtq']) / df['lctq']
df['op_margin'] = df['oiadpq'] / df['saleq']

# Normalize and combine signals
scaler = MinMaxScaler()
df[['debt_to_equity_norm', 'quick_ratio_norm', 'op_margin_norm']] = scaler.fit_transform(df[['debt_to_equity', 'quick_ratio', 'op_margin']])

weights = {'debt_to_equity': 0.4, 'quick_ratio': 0.3, 'op_margin': 0.3}
df['weighted_sum'] = df['debt_to_equity_norm'] * weights['debt_to_equity'] + \
                     df['quick_ratio_norm'] * weights['quick_ratio'] + \
                     df['op_margin_norm'] * weights['op_margin']

In [75]:
# Load returns data
returns_df = pd.DataFrame(engine.connect().execute(text('SELECT date,cusip,ticker,ret,prc,askhi,bidlo FROM returns')))
returns_df['ret'] = pd.to_numeric(returns_df['ret'], errors='coerce')
returns_df.dropna(subset=['ret'], inplace=True)
returns_df.rename(columns={'prc': 'Close', 'askhi': 'High', 'bidlo': 'Low'}, inplace=True)

In [76]:
# Merge the df and returns_df dataframes
merged_df = pd.merge(df, returns_df, left_on=['datadate', 'tic'], right_on=['date', 'ticker'])
merged_df = merged_df.sort_values(by='date')
merged_df = merged_df.set_index('date')

# Add an 'Open' column (assuming the open price is the same as the close price of the previous day)
merged_df['Open'] = merged_df['Close'].shift(1)

# Drop rows with missing data
merged_df.dropna(subset=['Open', 'High', 'Low', 'Close'], inplace=True)

In [87]:
# Calculate expanding quantiles outside the strategy and add them to the dataframe
quantile_80 = merged_df['weighted_sum'].expanding().quantile(0.8)
quantile_20 = merged_df['weighted_sum'].expanding().quantile(0.2)
merged_df['quantile_80'] = quantile_80
merged_df['quantile_20'] = quantile_20

class CombinedSignalStrategy(Strategy):
    def init(self):
        self.threshold = self.data.weighted_sum.mean()

    def next(self):
        if crossover(self.data.weighted_sum, self.threshold * 0.5):
            self.buy()
        elif crossover(self.threshold * 1.5, self.data.weighted_sum):
            self.sell()

In [88]:
merged_df

,datadate,gvkey,tic,cusip_x,dlttq,lseq,seqq,ltq,actq,invtq,...,weighted_sum,cusip_y,ticker,ret,Close,High,Low,Open,quantile_80,quantile_20
date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,2000-01-31,012379,AMWD,030506109,9.7330,155.8330,89.5070,66.3260,60.1490,22.5150,...,0.300905,03050610,AMWD,0.017857,17.8125,18.7500,17.750,28.4375,0.300905,0.300905
2000-01-31,2000-01-31,024925,ROP,776696106,153.6420,474.6490,239.4320,235.2170,172.7250,73.7950,...,0.310834,77669610,ROP,-0.030686,33.5625,34.4375,32.750,17.8125,0.308848,0.302891
2000-01-31,2000-01-31,025880,PDCO,703395103,1.3620,420.7210,311.4570,109.2640,312.5050,110.0530,...,0.315035,70339510,PDCO,-0.011348,43.5625,46.0000,43.000,33.5625,0.313355,0.304877
2000-01-31,2000-01-31,002436,CAL,129500104,162.0340,650.3380,249.9450,400.3930,487.7740,365.9890,...,0.297427,21079520,CAL,-0.059701,31.5000,33.5000,31.375,43.5625,0.312514,0.299514
2000-01-31,2000-01-31,004016,DG,256677105,1.2000,1450.9410,925.9210,525.0200,1095.5350,985.7150,...,0.298754,25666910,DG,0.021021,21.2500,21.5000,20.625,31.5000,0.311674,0.298489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,2023-11-30,010840,UNF,904708104,50.3340,2602.0520,2040.1730,561.8790,858.9380,398.1240,...,0.312403,90470810,UNF,0.013744,172.6000,172.7200,168.780,63.6600,0.316777,0.302906
2023-11-30,2023-11-30,011600,WOR,981811102,383.8320,3584.1370,1792.8090,1660.1700,1772.6030,576.4110,...,0.304203,98181110,WOR,0.008155,71.7000,71.8700,70.080,172.6000,0.316777,0.302906
2023-11-30,2023-11-30,003362,CAG,205887102,7493.3000,22120.0000,9004.0000,13045.8000,3469.1000,2277.6000,...,0.300574,20588710,CAG,0.009636,28.2900,28.3250,27.870,71.7000,0.316777,0.302906


In [98]:
from decimal import Decimal

# Check if the index is already a DatetimeIndex
if not isinstance(merged_df.index, pd.DatetimeIndex):
    merged_df.index = pd.to_datetime(merged_df.index)

# Calculate expanding quantiles outside the strategy and add them to the dataframe
quantile_80 = merged_df['weighted_sum'].expanding().quantile(0.8)
quantile_20 = merged_df['weighted_sum'].expanding().quantile(0.2)
merged_df['quantile_80'] = quantile_80
merged_df['quantile_20'] = quantile_20

# Drop rows with missing data
merged_df.dropna(subset=['Open', 'High', 'Low', 'Close', 'quantile_80', 'quantile_20'], inplace=True)

# Convert relevant columns to float64 to avoid type conflicts
merged_df['Close'] = merged_df['Close'].astype(float)
merged_df['High'] = merged_df['High'].astype(float)
merged_df['Low'] = merged_df['Low'].astype(float)
merged_df['Open'] = merged_df['Open'].astype(float)
merged_df['quantile_80'] = merged_df['quantile_80'].astype(float)
merged_df['quantile_20'] = merged_df['quantile_20'].astype(float)

class CombinedSignalStrategy(Strategy):
    def init(self):
        self.threshold = 0.31

    def next(self):
        if crossover(self.data.weighted_sum, self.threshold):
            self.buy()
        elif crossover(self.threshold, self.data.weighted_sum):
            self.sell()

# Run the backtest
bt = Backtest(merged_df, CombinedSignalStrategy, cash=10000, commission=0.002)
stats = bt.run()
print(stats)

# Generate Sharpe ratio and PnL graph
print(f"Sharpe Ratio: {stats['Sharpe Ratio']:.2f}")
# bt.plot()

AssertionError: 